In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS fleetops_project.gold;

In [0]:
%sql
CREATE OR REPLACE TABLE fleetops_project.gold.dim_caminhao AS
SELECT 
    truck_id AS caminhao_sk,
    plate AS placa,
    model AS modelo,
    capacity_kg AS capacidade,
    manufacture_year AS ano_fabricacao
FROM fleetops_project.silver.trucks;

In [0]:
%sql
CREATE OR REPLACE TABLE fleetops_project.gold.dim_motorista AS
SELECT 
    driver_id AS motorista_sk,
    name AS nome,
    cnh,
    cnh_type AS tipo_cnh
FROM fleetops_project.silver.drivers;

In [0]:
%sql

CREATE OR REPLACE TABLE fleetops_project.gold.fact_telemetria AS
SELECT 
    t.event_id,
    t.event_time AS data_hora,
    t.truck_id,
    
    COALESCE(da.assignment_sk, -1) AS alocacao_sk,
    COALESCE(da.driver_id, -1) AS motorista_fk,
    
    t.speed_kmh AS velocidade,
    t.engine_rpm AS rpm,
    t.fuel_level AS nivel_combustivel,
    t.engine_temp AS temperatura_motor,
    t.gps_latitude,
    t.gps_longitude,
    
    CASE WHEN t.speed_kmh > 100 THEN 1 ELSE 0 END AS flag_excesso_velocidade,
    CASE WHEN t.engine_rpm > 3500 THEN 1 ELSE 0 END AS flag_rotacao_alta,
    CASE WHEN t.engine_temp > 105 THEN 1 ELSE 0 END AS flag_superaquecimento

FROM fleetops_project.silver.iot_telemetry t
LEFT JOIN fleetops_project.silver.driver_assignments_scd da
    ON t.truck_id = da.truck_id
    AND t.event_time >= da.effective_start
    AND t.event_time < da.effective_end;
    
OPTIMIZE fleetops_project.gold.fact_telemetria ZORDER BY (truck_id, data_hora);

In [0]:
%sql
CREATE OR REPLACE TABLE fleetops_project.gold.agg_driver_performance_daily AS
SELECT
    motorista_fk,
    DATE(data_hora) as data_referencia,
    
    COUNT(CASE WHEN flag_excesso_velocidade = 1 THEN 1 END) as qtd_infracoes_velocidade,
    COUNT(CASE WHEN flag_rotacao_alta = 1 THEN 1 END) as qtd_abuso_motor,

    (MAX(nivel_combustivel) - MIN(nivel_combustivel)) as consumo_tanque_percentual,
    
    AVG(rpm) as rpm_medio_diario,
    MAX(velocidade) as velocidade_maxima_atingida

FROM fleetops_project.gold.fact_telemetria
GROUP BY 1, 2;